In [1]:
import numpy as np
import pandas as pd 

In [3]:
data = pd.read_csv('../../handledData.csv')

In [4]:
data.head()

,Unnamed: 0,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,trans_days_interval_filter,trans_days_interval,...,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
0,0,0.01,0.99,0,0.90,0.55,0.313,17.0,27.0,26.0,...,2900.0,1688.0,1200.0,75.0,1.0,2.0,1200.0,1200.0,12.0,18.0
1,1,0.02,0.94,2000,1.28,1.00,0.458,19.0,30.0,14.0,...,3500.0,1758.0,15100.0,80.0,5.0,6.0,22800.0,9360.0,4.0,2.0
2,2,0.04,0.96,0,1.00,1.00,0.114,13.0,68.0,22.0,...,1600.0,1250.0,4200.0,87.0,1.0,1.0,4200.0,4200.0,2.0,6.0
3,3,0.00,0.96,2000,0.13,0.57,0.777,22.0,14.0,6.0,...,3200.0,1541.0,16300.0,80.0,5.0,5.0,30000.0,12180.0,2.0,4.0
4,4,0.01,0.99,0,0.46,1.00,0.175,13.0,66.0,42.0,...,2300.0,1630.0,8300.0,79.0,2.0,2.0,8400.0,8250.0,22.0,120.0


In [7]:
data.drop(columns=['latest_query_time', 'loans_latest_time', 'first_transaction_time'], inplace=True)

In [9]:
# IV特征选择
def CalcIV(Xvar, Yvar):
    N_0 = np.sum(Yvar == 0)    # 非响应客户
    N_1 = np.sum(Yvar == 1)    # 响应客户
    N_0_group = np.zeros(np.unique(Xvar).shape)   # 分组

    N_1_group = np.zeros(np.unique(Xvar).shape)
    for i in range(len(np.unique(Xvar))):
        # 计算非响应客户和响应客户的各个组内的相关值
        N_0_group[i] = Yvar[(Xvar == np.unique(Xvar)[i]) & (Yvar == 0)].count()
        N_1_group[i] = Yvar[(Xvar == np.unique(Xvar)[i]) & (Yvar == 1)].count()
    # iv值
    iv = np.sum((N_0_group / N_0 - N_1_group / N_1) * np.log((N_0_group / N_0) / (N_1_group / N_1)))
    if iv >= 1.0:  # 处理极端值
        iv = 1
    return iv


def caliv_batch(df, Yvar):
    ivlist = []
    for col in df.columns:
        iv = CalcIV(df[col], Yvar)
        ivlist.append(iv)
    names = list(df.columns)
    iv_df = pd.DataFrame({'Var': names, 'Iv': ivlist}, columns=['Var', 'Iv'])
    return iv_df, ivlist

In [10]:
y = data['status']
X = data.drop(['status'], axis=1)
im_iv, ivl = caliv_batch(X, y)

c:\users\ly\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in true_divide
  del sys.path[0]
c:\users\ly\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in log
  del sys.path[0]


In [11]:
im_iv

,Var,Iv
0,Unnamed: 0,1.000000
1,low_volume_percent,1.000000
2,middle_volume_percent,1.000000
3,take_amount_in_later_12_month_highest,1.000000
4,trans_amount_increase_rate_lately,1.000000
5,trans_activity_month,1.000000
6,trans_activity_day,1.000000
7,transd_mcc,1.000000
8,trans_days_interval_filter,1.000000
9,trans_days_interval,1.000000


In [14]:
im_iv.Iv.describe()

count    81.000000
mean      0.916615
std       0.273108
min       0.000755
25%       1.000000
50%       1.000000
75%       1.000000
max       1.000000
Name: Iv, dtype: float64

In [15]:
threshold = 0.02
data_index = []
for i in range(len(ivl)):
    if im_iv['Iv'][i] < threshold:
        data_index.append(im_iv['Var'][i])
data.drop(data_index, axis=1, inplace=True)

In [16]:
data.describe()

,Unnamed: 0,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,trans_days_interval_filter,trans_days_interval,...,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
count,4455.000000,4455.000000,4455.000000,4455.000000,4455.000000,4455.000000,4455.000000,4455.000000,4455.000000,4455.000000,...,4455.000000,4455.000000,4455.000000,4455.000000,4455.000000,4455.000000,4455.000000,4455.000000,4455.000000,4455.000000
mean,2368.938272,0.021854,0.901556,1969.169473,14.887151,0.804644,0.367699,17.572166,28.947853,21.688664,...,3390.033670,1820.522110,9187.586981,76.041975,4.731762,5.226936,16153.535354,8008.078339,24.001355,55.170370
std,1373.466462,0.042197,0.143919,3995.427919,716.860005,0.196600,0.170310,4.483531,22.726473,16.515842,...,1474.147634,583.477472,7372.836703,14.540045,2.975142,3.409934,14304.131791,5680.624864,37.586470,53.492299
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.120000,0.033000,2.000000,0.000000,4.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.000000,-2.000000
25%,1174.500000,0.010000,0.880000,0.000000,0.610000,0.670000,0.236000,15.000000,16.000000,12.000000,...,2300.000000,1535.500000,4800.000000,77.000000,2.000000,3.000000,7800.000000,4737.000000,5.000000,10.000000
50%,2364.000000,0.010000,0.960000,600.000000,0.960000,0.860000,0.352000,18.000000,23.000000,17.000000,...,3100.000000,1810.000000,7700.000000,79.000000,4.000000,5.000000,13800.000000,7050.000000,14.000000,36.000000
75%,3557.500000,0.020000,0.990000,2000.000000,1.580000,1.000000,0.480000,20.000000,32.000000,26.000000,...,4300.000000,2100.000000,11700.000000,80.000000,7.000000,7.000000,20400.000000,10000.000000,24.000000,91.000000
max,4753.000000,1.000000,1.000000,68000.000000,47596.740000,1.000000,0.941000,42.000000,285.000000,234.000000,...,10000.000000,6900.000000,87100.000000,87.000000,18.000000,20.000000,266400.000000,82800.000000,360.000000,323.000000


随机森林

In [17]:
from sklearn.ensemble import RandomForestClassifier

In [18]:
feat_lables = X.columns
forest = RandomForestClassifier(n_estimators=10000, random_state=0, n_jobs=1)
forest.fit(X, y)
importance = forest.feature_importances_
imp_result = np.argsort(importance)[::-1]
for i in range(X.shape[1]):
    print("%2d. %-*s %f" % (i+1, 30, feat_lables[i], importance[imp_result[i]]))
threshold = 0.01
data_index = list(X.columns[importance < threshold])
print(X.shape)
X.drop(data_index, axis=1, inplace=True)
print(X.shape)

 1. Unnamed: 0                     0.050557
 2. low_volume_percent             0.048867
 3. middle_volume_percent          0.038664
 4. take_amount_in_later_12_month_highest 0.031810
 5. trans_amount_increase_rate_lately 0.026038
 6. trans_activity_month           0.023568
 7. trans_activity_day             0.017295
 8. transd_mcc                     0.017050
 9. trans_days_interval_filter     0.016989
10. trans_days_interval            0.016157
11. regional_mobility              0.016087
12. student_feature                0.015401
13. repayment_capability           0.015358
14. is_high_user                   0.015226
15. number_of_trans_from_2011      0.015095
16. historical_trans_amount        0.015032
17. historical_trans_day           0.014948
18. rank_trad_1_month              0.014911
19. trans_amount_3_month           0.014255
20. avg_consume_less_12_valid_month 0.014153
21. abs                            0.013982
22. top_trans_count_last_1_month   0.013727
23. avg_price_last_12